In [ ]:
'''web scraping using beautiful soap'''

import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
    response = requests.get(url, timeout=10)
    soup = BeautifulSoup(response.text, "html.parser")
    text = " ".join([p.get_text() for p in soup.find_all("p")])
    return text.lower()


url = "https://www.baunfire.com/"
text = extract_text_from_url(url)
f=open('markdown4.txt','w',encoding="utf-8")
f.write(text)
f.close()
automate=Automation('markdown4.txt')
response = automate.prediction()
print("########################################")
print(response)

print(text)  


In [ ]:
# prompt 1 (2/3)
"""Analyze the following website content and determine if it belongs to a Digital Marketing Agency.

        Look for similar keywords: Services, Web Design, Web Development, SEO Agency, Ads Agency, Digital, Marketing , Agency, Website Creation,
        brands,web,design,Boosts creativity,Reduces costs.
        
        If most of the keywords appear in the website content then classify it as *"Digital Marketing Agency"*.
        Otherwise classify it as *"Not a Digital Marketing Agency"* and format output as given below.
        No explanations.

       format the output as a json object with two field as  ***(status  "accepted" :  |"rejected") and (reason : )***
       give reason.
        Website Content:
        {website_content}

        Final Answer:
        """

In [3]:
import csv
import json
import re

# Simulated response text
response_text = """
{
  "status": "rejected",
  "reason": "The website content does not mention any digital marketing services, web design, web development, SEO agency, ads agency, digital, marketing, agency, website creation, or similar keywords."
}
"""

# Extract JSON object from text
match = re.search(r'\{.*\}', response_text, re.DOTALL)  # Finds the JSON part
if match:
    json_text = match.group(0)  # Extract the JSON string
    data = json.loads(json_text)  # Convert to dictionary

    # Save to CSV
    csv_file = "output.csv"
    with open(csv_file, mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Status", "Reason"])  # Writing header
        writer.writerow([data["status"], data["reason"]])  # Writing data

    print(f"Data saved to {csv_file}")
else:
    print("No JSON found in the response")


Data saved to output.csv


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# # Load GPT-2 model and tokenizer
# model_name = "gpt2"
# model = AutoModelForCausalLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")


# Read website content
with open("markdown1.txt", "r", encoding="utf-8") as f:
    website_content = f.read()

prompt = f"""
Analyze the following website content and determine if it belongs to a Digital Marketing Agency.

Look for these keywords: Services, Web Design, Web Development, SEO Agency, Ads Agency, Digital Marketing Agency, Agency, Website Creation.

If ALL keywords appear multiple times then classify it as **"Digital Marketing Agency"**.
Otherwise classify it as **"Not a Digital Marketing Agency"**
No explanations.



Website Content:
{website_content[:1000]}  # Limit text length to fit token limit

Final Answer:

"""
#Final Answer (ONLY output "Digital Marketing Agency" or "Not a Digital Marketing Agency")
# **🔹 Tokenize input**
inputs = tokenizer(prompt, return_tensors="pt")

# **🔹 Generate response from the model**
output = model.generate(**inputs)

# **🔹 Decode generated text**
response = tokenizer.decode(output[0], skip_special_tokens=True)

# Extract only the classification result

# print("Final Classification:", final_output)

lines = [line.strip() for line in response.split("\n") if line.strip()]

# Get the last non-empty line
if lines:
    final_output = lines[-1]  # Last valid sentence
else:
    final_output = "No classification result found."

# print("Final Classification:", final_output)

print(response)



In [ ]:
#chat gpt mail

import smtplib

from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import config

SENDER_EMAIL = config.MAIL
SENDER_PASSWORD = config.PASSWORD 
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

# Send email function
def send_email(recipient_email,status,reason):

    if status == "accepted":
        subject = "Your Website is Approved!"
        message_body = f"""
        Dear User,

        Congratulations! Your website has been classified as a **Digital Marketing Agency**.

        You can proceed with the next steps.

        Best Regards,
        Your Team
        """
    else:
        subject = "Your Website is Rejected"
        message_body = f"""
        Dear User,

        Unfortunately, your website **does not meet the criteria** for a Digital Marketing Agency.
        
        Reason: {reason}

        If you believe this is a mistake, please contact us.

        Best Regards,
        Your Team
        """


    msg = MIMEMultipart()
    msg["From"] = SENDER_EMAIL
    msg["To"] = recipient_email
    msg["Subject"] = subject

    msg.attach(MIMEText(message_body, "plain"))

    try:
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(SENDER_EMAIL, SENDER_PASSWORD)
        server.sendmail(SENDER_EMAIL, recipient_email, msg.as_string())
        server.quit()
        print(f"Email sent to {recipient_email}")
    except Exception as e:
        print(f" Error sending email: {e}")





In [ ]:
self.prompt = PromptTemplate(
            input_variables=[self.website_content],
            template="""
        Analyze the following website content and determine if it belongs to a Digital Marketing Agency.

        Look for similar keywords: Services, Web Design, Web Development, SEO Agency, Ads Agency, Digital, Marketing , Agency, Website Creation,
        brands,web,design,Boosts creativity,Reduces costs.

        ***The website content may not mention all the keywords. Use logic and find keywords similar to the above.****

        **if the above keywords or similar keywords are found then it can be a digital marketing agency**
       
       format the output as a json object with two field as  ***(status  "accepted" :  |"rejected") and (reason : )***
       give reason.
       ***Ensure proper json format with opening and closing brackets***
        ***Dont give explanation. Just display only the json object.***

        
      
        Website Content:
        {website_content}

        Final Answer:
        """
        )

In [ ]:
import asyncio
from crawl4ai import *
from chain import Automation
import json
import csv
import re
import os
import mail
'''
https://www.digitalsilk.com/
https://www.baunfire.com/
https://fourbynorth.com/
https://www.langchain.com/langchain
https://huggingface.co/
 '''

async def main():
    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(
            url="https://fourbynorth.com/",
        )
        # print(result.markdown)
        f=open('markdown4.txt','w',encoding="utf-8")
        f.write(result.markdown)
        f.close()
        automate=Automation('markdown4.txt')

        response = automate.prediction()
        print("########################################")
        print(response)
        # response = response.strip()  # Remove extra spaces/newlines

        # # Ensure the response starts with '{' and ends with '}'
        # if not response.startswith("{"):
        #     response = "{" + response
        # if not response.endswith("}"):
        #     response = response + "}"
                
        # url="https://huggingface.co/"
        # email='aswinshibu2024gcsj@gmail.com'      
        # # Extract JSON object from text
        # match = re.search(r'\{.*\}', response, re.DOTALL)  # Finds the JSON part
        # if match:
        #     json_text = match.group(0)  # Extract the JSON string
        #     data = json.loads(json_text)  # Convert to dictionary
            
        #     csv_file = "output.csv"
        #     write_header = not os.path.exists(csv_file) or os.stat(csv_file).st_size == 0
            
            
        #     with open(csv_file, mode="a", newline="") as file:
        #         writer = csv.writer(file)
        #         if write_header:
        #             writer.writerow(["Email","url","Status", "Reason"])
        #         # writer.writerow(["Status", "Reason"])  # Writing header
        #         writer.writerow([email,url,data["status"], data["reason"]])  # Writing data

        #     print(f"Data saved to {csv_file}")
        # else:
        #     print("No JSON found in the response")

        # mail.send_email(email,url,data["status"],data["reason"])

if __name__ == "__main__":
    asyncio.run(main())